In [1]:
def find_spans (text, layer_name):
    layer = getattr(text, layer_name)
    spancount = len(layer.spans)
    spanindexes = []
    text=text.text
    attributes = layer.attributes    
    
    for s in layer.spans:
        spanstart = s.start
        spanend = s.end
        spanindexes.append(spanstart)
        spanindexes.append(spanend)
    
    html_spans=[]
    
    spanindexes.sort()
    doubled=[]
    for i in range(len(spanindexes)-1):
        if spanindexes[i]==spanindexes[i+1]:
            doubled.append(spanindexes[i])
    for val in doubled:
        spanindexes.remove(val)
    
    for i in range(len(spanindexes)-1):
        if text[spanindexes[i]:spanindexes[i+1]]!=' ':
            html_spans.append([spanindexes[i],spanindexes[i+1]])
            
    printable=[]
    i = 0
    
    for html_span in html_spans:
        printable.append([])
        for s in layer.spans:
            if html_span[0] in range(s.start,s.end):
                for attribute in attributes:
                    printable[i].append(getattr(s,attribute))
        i+=1

    assert len(html_spans)==len(printable)
    
    i=0
    for attributes in printable:
        if len(attributes)==0:
            printable.remove(attributes)
            html_spans.remove(html_spans[i])
        i+=1

    return html_spans,printable

In [2]:
def add_spans(text,layer):
    t = text.text

    spans = find_spans(text,layer)[0]

    contents = find_spans(text,layer)[1]
    attributes = getattr(text,layer).attributes
    fragments = []
    last_end = 0
    new_contents = []
    
    for content in contents:
        new_content=[]
        for i in range(len(attributes)):
            new_content.append(''.join([attributes[i],": ",str(content[i])]))
            new_contents.append(new_content)
            
    #contents = new_contents
    
    i = 0  
    for s, e in spans:
        fragments.append(t[last_end:s])
        fragments.append('<span onclick="box(')
        fragments.append('`')
        fragments.append(str(contents[i]))
        fragments.append('`)')
        fragments.append('">')
        fragments.append(t[s:e])
        fragments.append('</span>')
        last_end = e
        i+=1
    fragments.append(t[e:])
    return ''.join(fragments)

In [3]:
t = '''
<html>
<style>
.boxes {{
    background: linear-gradient(to bottom right, white, lightgray);
    border: 1px solid black;
    border-radius: 4px;
    box-shadow: 4px 4px 4px gray;
    width: fit-content;
}}
</style>
<body>

<p id="maintext">{}</p>

<script>

function box(content) {{
    let textbox = document.createElement("DIV");
    textbox.classList.add("boxes");
    let t = document.createTextNode(content);
    textbox.appendChild(t);
    let textNode = document.getElementById("maintext");
    let previousBox = document.querySelector('.boxes');
    if (previousBox){{
        previousBox.parentNode.removeChild(previousBox);
    }}
    textNode.appendChild(textbox);
}}

</script>

</body>
</html>
'''


In [4]:
from IPython.display import display_html
from estnltk.tests import new_text
from estnltk import Text

#a=Text("Tere, maailm!")
#a.tag_layer(["morph_analysis"])

def display_layer(text,layer):
    result=add_spans(text,layer)
    display_html(t.format(result), raw=True)
    
display_layer(new_text(5),"layer_1")


Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd tuhat.